정규화를 해서 dt에 넣어보기 -> 모든 데이터를 0~1 사이의 값이 될 수 있또록
lead_time, avg_price_per_room -> cate ->  num으로 바꾸기


In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV


In [49]:

train = pd.read_csv('conti_to_dis_train.csv')
train.keys()

Index(['Booking_ID', 'no_of_adults', 'no_of_children', 'no_of_weekend_nights',
       'no_of_week_nights', 'type_of_meal_plan', 'required_car_parking_space',
       'room_type_reserved', 'lead_time', 'arrival_year', 'arrival_month',
       'arrival_date', 'market_segment_type', 'repeated_guest',
       'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
       'avg_price_per_room', 'no_of_special_requests', 'booking_status',
       'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
       'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
       'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
       'Complementary', 'Online', 'Offline', 'Aviation',
       'lead_time_under_iqr_1', 'lead_time_under_iqr_2',
       'lead_time_under_iqr_3', 'lead_time_upper_iqr_3', 'price_under_iqr_1',
       'price_under_iqr_2', 'price_under_iqr_3', 'price_upper_iqr_3'],
      dtype='object')

In [62]:


'''
feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests',
                 'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
                 'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
                 'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
                 'Complementary', 'Online', 'Offline', 'Aviation']
'''


train = pd.read_csv('onehot_sample.csv')
#no_of_adults,no_of_children,avg_price_per_room 특징만 고려
# feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
#                  'no_of_week_nights', 'required_car_parking_space',
#                  'lead_time', 'arrival_year', 'arrival_month',
#                  'arrival_date', 'repeated_guest',
#                  'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
#                  'avg_price_per_room', 'no_of_special_requests',
#                  'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
#                  'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
#                  'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
#                  'Complementary', 'Online', 'Offline', 'Aviation']\

feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests',
                 'meal_type_1', 'meal_type_2', 'meal_type_3', 'meal_type_4',
                 'room_type_1', 'room_type_2', 'room_type_3', 'room_type_4',
                 'room_type_5', 'room_type_6', 'room_type_7', 'Corporate',
                 'Complementary', 'Online', 'Offline', 'Aviation']

X = train[feature_names]
y = train["booking_status"]

# Decision Tree 학습/예측/평가
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Decision Tree 학습/예측/평가
dt_model = DecisionTreeClassifier(max_depth = 10, min_samples_leaf = 2, criterion="gini", min_samples_split = 2)
scores = cross_val_score(dt_model, X, y, cv = 10)
print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())
print("Number of CV Scores used in Average: ", len(scores))

Cross Validation Scores:  [0.86417323 0.87480315 0.87908625 0.87554155 0.86648287 0.87593541
 0.86096889 0.87239071 0.87199685 0.86687672]
Average CV Score:  0.8708255621749528
Number of CV Scores used in Average:  10


In [52]:

def get_dt_graph(dt_classifier):
    fig = plt.figure(figsize=(25, 20))
    _ = dt_model.plot_tree(dt_classifier,
                       feature_names=X.columns,
                       class_names=['No Disease', "Disease"],
                       filled=True)


def evaluate_model(dt_classifier):
    print("Train Accuracy :", accuracy_score(
        y_train, dt_classifier.predict(X_train)))
    print("Train Confusion Matrix:")
    print(confusion_matrix(y_train, dt_classifier.predict(X_train)))
    print("-"*50)
    print("Test Accuracy :", accuracy_score(
        y_test, dt_classifier.predict(X_test)))
    print("Test Confusion Matrix:")
    print(confusion_matrix(y_test, dt_classifier.predict(X_test)))


params = {
    'max_depth' : [2, 4, 6, 8, 10], 
    'min_samples_leaf' : [2, 4, 6, 8, 10], 
    'criterion' : ["gini", "entropy"],
    'min_samples_split' : [2, 4, 6, 8, 10]
}


# Instantiate the grid search model
grid_search = GridSearchCV(estimator=dt_model,
                           param_grid=params,
                           cv=10, n_jobs=-1, verbose=1, scoring="accuracy")

grid_search.fit(X_train, y_train)


Fitting 10 folds for each of 250 candidates, totalling 2500 fits


GridSearchCV(cv=10,
             estimator=DecisionTreeClassifier(max_depth=10, min_samples_leaf=2),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [2, 4, 6, 8, 10],
                         'min_samples_split': [2, 4, 6, 8, 10]},
             scoring='accuracy', verbose=1)

In [53]:
GridSearchCV(cv = 10, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_samples_split': [5, 10, 20, 50, 100]},
             scoring='accuracy', verbose=1)

score_df = pd.DataFrame(grid_search.cv_results_)
# score_df.to_csv('dt_model_grid_search.csv')
score_df.nlargest(5, "mean_test_score")


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
104,0.082493,0.004759,0.004028,0.001257,gini,10,2,10,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.788451,...,0.777953,0.781513,0.793067,0.782038,0.786765,0.805147,0.785189,0.786338,0.007786,1
103,0.078654,0.004883,0.003204,0.001168,gini,10,2,8,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.790026,...,0.777428,0.781513,0.792017,0.782038,0.786765,0.804097,0.784664,0.786338,0.007456,2
102,0.076821,0.003318,0.004902,0.002427,gini,10,2,6,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.789501,...,0.776903,0.782563,0.792542,0.782038,0.786239,0.804622,0.783613,0.786285,0.007575,3
100,0.080340,0.002452,0.003303,0.000676,gini,10,2,2,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.790551,...,0.777953,0.780987,0.793592,0.782563,0.786239,0.803571,0.783613,0.786128,0.007498,4
107,0.082456,0.007901,0.003701,0.000640,gini,10,4,6,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.791601,...,0.779003,0.778887,0.793067,0.779937,0.787815,0.801471,0.786765,0.786127,0.007098,5


In [63]:

dt_model.fit(X_train, y_train)


DecisionTreeClassifier(max_depth=10, min_samples_leaf=2)

In [64]:
y_pred = dt_model.predict(X_test)
print('Accuracy: {:.2f}'.format(accuracy_score(y_test, y_pred)))
evaluate_model(dt_model)

Accuracy: 0.87
Train Accuracy : 0.8873660995589162
Train Confusion Matrix:
[[11961   873]
 [ 1272  4938]]
--------------------------------------------------
Test Accuracy : 0.8724007561436673
Test Confusion Matrix:
[[3959  321]
 [ 489 1579]]


In [24]:
from sklearn.preprocessing import OneHotEncoder

real_test = pd.read_csv('test.csv')

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['type_of_meal_plan']])
train_cat = pd.DataFrame(train_cat)
real_test['meal_type_1'] = train_cat.loc[:, 0]
real_test['meal_type_2'] = train_cat.loc[:, 1]
real_test['meal_type_3'] = train_cat.loc[:, 2]
real_test['meal_type_4'] = train_cat.loc[:, 3]

ohe = OneHotEncoder(sparse=False)
# fit_transform은 train에만 사용하고 test에는 학습된 인코더에 fit만 해야한다
train_cat = ohe.fit_transform(real_test[['room_type_reserved']])
train_cat = pd.DataFrame(train_cat)
real_test['Room_Type_1'] = train_cat.loc[:, 0]
real_test['Room_Type_2'] = train_cat.loc[:, 1]
real_test['Room_Type_3'] = train_cat.loc[:, 2]
real_test['Room_Type_4'] = train_cat.loc[:, 3]
real_test['Room_Type_5'] = train_cat.loc[:, 4]
real_test['Room_Type_6'] = train_cat.loc[:, 5]
real_test['Room_Type_7'] = train_cat.loc[:, 6]

train_cat = ohe.fit_transform(real_test[['market_segment_type']])
train_cat = pd.DataFrame(train_cat)
real_test['Corporate'] = train_cat.loc[:, 0]
real_test['Complementary'] = train_cat.loc[:, 1]
real_test['Online'] = train_cat.loc[:, 2]
real_test['Offline'] = train_cat.loc[:, 3]
real_test['Aviation'] = train_cat.loc[:, 4]

real_test.to_csv('onehot_test.csv', index=False)



In [33]:
real_test = pd.read_csv('onehot_test.csv')

feature_names = ['no_of_adults', 'no_of_children', 'no_of_weekend_nights',
                 'no_of_week_nights', 'required_car_parking_space',
                 'lead_time', 'arrival_year', 'arrival_month',
                 'arrival_date', 'repeated_guest',
                 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled',
                 'avg_price_per_room', 'no_of_special_requests', 'meal_type_1',
                 'meal_type_2', 'meal_type_3', 'meal_type_4', 'Room_Type_1',
                 'Room_Type_2', 'Room_Type_3', 'Room_Type_4', 'Room_Type_5',
                 'Room_Type_6', 'Room_Type_7', 'Corporate', 'Complementary', 'Online',
                 'Offline', 'Aviation']

test = real_test[feature_names]
y_pred = dt_model.predict(test)

sample = pd.DataFrame()
sample['Booking_ID'] = real_test['Booking_ID']
sample['booking_status'] = y_pred
sample.to_csv('sample.csv', index=False)
# real_test.keys()


c:\Users\sksoh\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- Room_Type_1
- Room_Type_2
- Room_Type_3
- Room_Type_4
- Room_Type_5
- ...
Feature names seen at fit time, yet now missing:
- room_type_1
- room_type_2
- room_type_3
- room_type_4
- room_type_5
- ...

  warnings.warn(message, FutureWarning)
